In [ ]:
"""
Algorithm Description:

This Algorithm replicates our Trading Strategy for the Bloomberg Trading Challenge. It creates 7 filters as laid out in 
in the paper and invests if the criteria is fulfilled. Furthermore the selection is balanced according to the sector
balancing as described in the paper. 

"""

import quantopian.algorithm as algo
from quantopian.pipeline import Pipeline
from quantopian.algorithm import order_optimal_portfolio
from quantopian.algorithm import attach_pipeline, pipeline_output
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage,  SimpleBeta, AnnualizedVolatility
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.classifiers.fundamentals import Sector
from quantopian.pipeline import CustomFactor
import quantopian.optimize as opt
import pandas as pd
import numpy as np

def initialize(context):
    # Schedule our rebalance function to run at the start of
    # every day when the market opens
    schedule_function(
        my_rebalance,
        date_rules.every_day(),  
        time_rules.market_open()
    )

    # Create our pipeline and attach it to our algorithm.
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe, 'my_pipeline')

def make_pipeline():
    """
    Create our pipeline.
    """

    universe = QTradableStocksUS()

    # Filter 1: EPS has to be larger than zero
    eps = Fundamentals.basic_eps_earnings_reports.latest
    eps_filter = eps > 0

    #Filter 2: Market cap has to be larger than 2 billion USD (2'000'000'000)
    market_cap = Fundamentals.market_cap.latest
    market_cap_filter = market_cap.percentile_between(20,100)

    #Filter 3 and 4: Moving Average (MAVG) 10 days > 20 days AND 20 days > 100 days
    ma100 = SimpleMovingAverage(inputs = [USEquityPricing.close], window_length = 100, mask= universe)
    ma20 = SimpleMovingAverage(inputs = [USEquityPricing.close], window_length = 20, mask= universe)
    ma10 = SimpleMovingAverage(inputs = [USEquityPricing.close], window_length = 10, mask= universe)


    long_ma_filter = ma20 > ma100
    short_ma_filter = ma10 > ma20
    ma_filter = short_ma_filter & long_ma_filter

    #Filter 5: 1y Beta against SPY has to be below 1 (defensive stocks only)
    #SPY = symbols('SPY') ; seems like this doesnt work as it returns SPY as a list, we use sid(8554)                               now to connect to SPY 
    
    beta = SimpleBeta(target=sid(8554), regression_length= 252)
    beta_filter = beta < 1

    #Filter 6: Yearly Volatility has to be below the 85 percentile of all assets in the universe
    vola = AnnualizedVolatility(window_length=252)
    vola_filter = vola.percentile_between(0,85)

    #Filter 7: PE ratio has to be smaller than 15
    #pe_ratio = Fundamentals.pe_ratio.latest
    #pe_filter = pe_ratio < 25

    #Filter 8: Z-Score of PE ratio has to be below zero (below the average) 
    #Creating the mean of PE grouped by sector for zscore
    class IndustryMeanPE(CustomFactor):  
        inputs = [Fundamentals.pe_ratio, Fundamentals.morningstar_sector_code]  
        window_length = 252

        def compute(self, today, assets, out, pe, industry_codes):  
            df = pd.DataFrame(index=assets, data={"pe_ratio": pe[-1], "industry_codes": industry_codes[-1]}) 

            out[:] = df.groupby("industry_codes").transform(np.mean).values.flatten()
        
    #Creating the standard deviation of the PE grouped by sector for zscore
    class industry_std(CustomFactor):  
        inputs = [Fundamentals.pe_ratio, Fundamentals.morningstar_sector_code]  
        window_length = 252

        def compute(self, today, assets, out, pe, industry_codes):  
            df = pd.DataFrame(index=assets, data={"pe_ratio": pe[-1], "industry_codes": industry_codes[-1]}) 

            out[:] = df.groupby("industry_codes").transform(np.nanstd).values.flatten()
        
    #Getting PE ratio and calulating zscore
    pe = Fundamentals.pe_ratio.latest
    industry_pe = IndustryMeanPE()
    industry_std_of_pe = industry_std()
    zscore = (pe - industry_pe)/industry_std_of_pe
    
    #We only want below average PE ratios so zscore < 0
    zscore_filter = zscore <0
    
    #All combined
    positive_ma = eps_filter & market_cap_filter & ma_filter & beta_filter & vola_filter & zscore_filter
    
    industry = Fundamentals.morningstar_sector_code.latest
    industry_101 = ~(industry != 101)
    positive_ma_101 = industry_101 & positive_ma
    industry_102 = ~(industry != 102)
    positive_ma_102 = industry_102 & positive_ma
    industry_103 = ~(industry != 103)
    positive_ma_103 = industry_103 & positive_ma
    industry_104 = ~(industry != 104)
    positive_ma_104 = industry_104 & positive_ma
    industry_205 = ~(industry != 205)
    positive_ma_205 = industry_205 & positive_ma
    industry_206 = ~(industry != 206)
    positive_ma_206 = industry_206 & positive_ma
    industry_207 = ~(industry != 207)
    positive_ma_207 = industry_207 & positive_ma
    industry_308 = ~(industry != 308)
    positive_ma_308 = industry_308 & positive_ma
    industry_309 = ~(industry != 309)
    positive_ma_309 = industry_309 & positive_ma
    industry_310 = ~(industry != 310)
    positive_ma_310 = industry_310 & positive_ma
    industry_311 = ~(industry != 311)
    positive_ma_311 = industry_311 & positive_ma

    # Filter for all securities that we want to trade.

    return Pipeline(
        columns={
            'positive_ma': positive_ma,
            'positive_ma_101': positive_ma_101,
            'positive_ma_102': positive_ma_102,
            'positive_ma_103': positive_ma_103,
            'positive_ma_104': positive_ma_104,
            'positive_ma_205': positive_ma_205,
            'positive_ma_206': positive_ma_206,
            'positive_ma_207': positive_ma_207,
            'positive_ma_308': positive_ma_308,
            'positive_ma_309': positive_ma_309,
            'positive_ma_310': positive_ma_310,
            'positive_ma_311': positive_ma_311,

            
        },
        screen=(positive_ma),
    )

def compute_target_weights(context, data):
    """
    Compute ordering weights.
    """
    count_101=len(context.positive_ma_101)
    count_102=len(context.positive_ma_102)
    count_103=len(context.positive_ma_103)
    count_104=len(context.positive_ma_104)
    count_205=len(context.positive_ma_205)
    count_206=len(context.positive_ma_206)
    count_207=len(context.positive_ma_207)
    count_308=len(context.positive_ma_308)
    count_309=len(context.positive_ma_309)
    count_310=len(context.positive_ma_310)
    count_311=len(context.positive_ma_311)

#Industry filter
    #101: 'Basic Materials',
    #102: 'Consumer Cyclical',
    #103: 'Financial Services',
    #104: 'Real Estate',
    #205: 'Consumer Defensive',
    #206: 'Healthcare',
    #207: 'Utilities',
    #308: 'Communication Services',
    #309: 'Energy',
    #310: 'Industrials',
    #311: 'Technology' ,


     #Here we want to compute our ordering weight
    weight_sector = 1/10
    max_weight_stock = weight_sector / 2
    #101
    
    nr_101 = count_101
    if nr_101 == 0:
        weight_101 =0
    else:
        weight_101 = (weight_sector-(1/20)) / nr_101
        if weight_101 > max_weight_stock: 
            weight_101 = max_weight_stock
        else:
            weight_101 = weight_101
    #102

    nr_102 = count_102
    if nr_102 == 0:
        weight_102 = 0
    else:
        weight_102 = (weight_sector-(1/20)) / nr_102
        if weight_102 > max_weight_stock: 
            weight_102 = max_weight_stock
        else:
            weight_102 = weight_102


    nr_103 = count_103
    if nr_103 == 0:
        weight_103 = 0
    else:
        weight_103 = (weight_sector-(1/20)) / nr_103
        if weight_103 > max_weight_stock: 
            weight_103 = max_weight_stock
        else:
            weight_103 = weight_103
    #104 
    nr_104 = count_104
    if nr_104 == 0:
        weight_104 = 0
    else:
        weight_104 = (weight_sector+(1/20)) / nr_104
        if weight_104 > max_weight_stock: 
            weight_104 = max_weight_stock
        else:
            weight_104 = weight_104


    #205 

    nr_205 = count_205
    if nr_205 == 0:
        weight_205 = 0
    else:
        weight_205 = (weight_sector+(1/20)) / nr_205
        if weight_205 > max_weight_stock: 
            weight_205 = max_weight_stock
        else:
            weight_205 = weight_205
    #206 
    nr_206 = count_206
    if nr_206 == 0:
        weight_206 = 0
    else:
        weight_206 = (weight_sector+(1/20)) / nr_206
        if weight_206 > max_weight_stock: 
            weight_206 = max_weight_stock
        else:
            weight_206 = weight_206


    #207 

    nr_207 = count_207
    if nr_207 == 0:
        weight_207 = 0
    else:
        weight_207 = (weight_sector) / nr_207
        if weight_207 > max_weight_stock: 
            weight_207 = max_weight_stock
        else:
            weight_207 = weight_207


    #308 

    nr_308 = count_308
    if nr_308 == 0:
        weight_308 = 0
    else:
        weight_308 = (weight_sector) / nr_308
        if weight_308 > max_weight_stock: 
            weight_308 = max_weight_stock
        else:
            weight_308 = weight_308


    #309 

    nr_309 = count_309
    if nr_309 == 0:
        weight_309 = 0
    else:
        weight_309 = (weight_sector-(1/10)) / nr_309
        if weight_309 > max_weight_stock: 
            weight_309 = max_weight_stock
        else:
            weight_309 = weight_309


    #310 

    nr_310 = count_310
    if nr_310 == 0:
        weight_310 = 0
    else:
        weight_310 = (weight_sector) / nr_310
        if weight_310 > max_weight_stock: 
            weight_310 = max_weight_stock
        else:
            weight_310 = weight_310
    #311 

    nr_311 = count_311
    if nr_311 == 0:
        weight_311 = 0
    else:
        weight_311 = (weight_sector+(1/20)) / nr_311
        if weight_311 > max_weight_stock: 
            weight_311 = max_weight_stock
        else:
            weight_311 = weight_311 
    
    
    # Initialize empty target weights dictionary.
    # This will map securities to their target weight.
    weights = {}

    if context.positive_ma_101:
        weight_101 = weight_101
    elif context.positive_ma_102:
        weight_102 = weight_102
    elif context.positive_ma_103:
        weight_103 = weight_103
    elif context.positive_ma_104:
        weight_104 = weight_104
    elif context.positive_ma_205:
        weight_205 = weight_205
    elif context.positive_ma_206:
        weight_206 = weight_206
    elif context.positive_ma_207:
        weight_207 = weight_207
    elif context.positive_ma_308:
        weight_308 = weight_308
    elif context.positive_ma_309:
        weight_309 = weight_309
    elif context.positive_ma_310:
        weight_310 = weight_310
    elif context.positive_ma_311:
        weight_311 = weight_311
        
    else:
        return weights
    # Exit positions in our portfolio if they are not
    # in our longs or shorts lists.
    for security in context.portfolio.positions:
        if security not in context.positive_ma_101 and security not in context.positive_ma_102 and security not in context.positive_ma_103 and security not in context.positive_ma_104 and security not in context.positive_ma_205 and security not in context.positive_ma_206 and security not in context.positive_ma_207 and security not in context.positive_ma_308 and security not in context.positive_ma_309 and security not in context.positive_ma_310 and security not in context.positive_ma_311  and data.can_trade(security):
            weights[security] = 0

    for security in context.positive_ma_101:  
        weights[security] = weight_101
    for security in context.positive_ma_102:  
        weights[security] = weight_102
    for security in context.positive_ma_103:  
        weights[security] = weight_103
    for security in context.positive_ma_104:  
        weights[security] = weight_104    
    for security in context.positive_ma_205:  
        weights[security] = weight_205
    for security in context.positive_ma_206:  
        weights[security] = weight_206
    for security in context.positive_ma_207:  
        weights[security] = weight_207
    for security in context.positive_ma_308:  
        weights[security] = weight_308
    for security in context.positive_ma_309:  
        weights[security] = weight_309
    for security in context.positive_ma_310:  
        weights[security] = weight_310
    for security in context.positive_ma_311:  
        weights[security] = weight_311
    
    return weights

def before_trading_start(context, data):
    """
    Get pipeline results.
    """

    # Gets our pipeline output every day.
    pipe_results = pipeline_output('my_pipeline')

    # Go long in securities for which the 'longs' value is True,
    # and check if they can be traded.
    context.positive_ma_101 = []
    for sec in pipe_results[pipe_results['positive_ma_101']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_101.append(sec)
    context.positive_ma_102 = []
    for sec in pipe_results[pipe_results['positive_ma_102']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_102.append(sec)    
    context.positive_ma_103 = []
    for sec in pipe_results[pipe_results['positive_ma_103']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_103.append(sec)
    context.positive_ma_104 = []
    for sec in pipe_results[pipe_results['positive_ma_104']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_104.append(sec)    
    context.positive_ma_205 = []
    for sec in pipe_results[pipe_results['positive_ma_205']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_205.append(sec)
    context.positive_ma_206 = []
    for sec in pipe_results[pipe_results['positive_ma_206']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_206.append(sec)    
    context.positive_ma_207 = []
    for sec in pipe_results[pipe_results['positive_ma_207']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_207.append(sec)
    context.positive_ma_308 = []
    for sec in pipe_results[pipe_results['positive_ma_308']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_308.append(sec)    
    context.positive_ma_309 = []
    for sec in pipe_results[pipe_results['positive_ma_309']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_309.append(sec)
    context.positive_ma_310 = []
    for sec in pipe_results[pipe_results['positive_ma_310']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_310.append(sec)
    context.positive_ma_311 = []
    for sec in pipe_results[pipe_results['positive_ma_311']].index.tolist():
        if data.can_trade(sec):
            context.positive_ma_311.append(sec)

def my_rebalance(context, data):
    """
    Rebalance weekly.
    """
    target_weights = compute_target_weights(context, data)
    if target_weights:
        order_optimal_portfolio(
            objective=opt.TargetWeights(target_weights),
            constraints=[],
        )